In [2]:
% matplotlib inline

from scikits.talkbox.features import mfcc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pyaudio as pa
import re
import os
from collections import defaultdict
from collections import Counter

random_state=24

音声ファイルimport

In [3]:
files = []
datapath = '../cry_data/devided_wav_data/'

for x in os.listdir(datapath):  
    if '.wav' in x:
        files.append(x)

mfccで特徴量とラベルのセット作成

In [9]:
label_dict = {'hu':0, 'ti':1, 'dc':2}
X_dict = defaultdict(list)
y_dict = defaultdict(list)
for f in files:
    x, sample_rate = sf.read(datapath + f)
    x = np.clip(x, 1e-10, 1)
    ceps,mspec,spec = mfcc(x, nwin=256, nfft=512, fs=8000, nceps=13)
    if '-hu.' in f:
        for cep in ceps:
            X_dict[re.sub(r'^\d*-', '', f)].append(cep)
            y_dict[re.sub(r'^\d*-', '', f)].append(label_dict['hu'])
    elif '-ti.' in f:
        for cep in ceps:
            X_dict[re.sub(r'^\d*-', '', f)].append(cep)
            y_dict[re.sub(r'^\d*-', '', f)].append(label_dict['ti'])
    else:
        for cep in ceps:
            X_dict[re.sub(r'^\d*-', '', f)].append(cep)
            y_dict[re.sub(r'^\d*-', '', f)].append(label_dict['dc'])
X_dict = dict(X_dict)
y_dict = dict(y_dict)
for k, v in X_dict.items():
    X_dict[k] = np.array(v)
for k, v in y_dict.items():
    y_dict[k] = np.array(v)

分類器で学習、評価

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

In [34]:
precision = 0
recall = 0
f_value = 0

n_splits = 7

skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
file_list = np.array(list(X_dict.keys()))
X = np.array(list(X_dict.values()))
y = np.array(list(y_dict.values()))
y_sample = np.array([a[0] for a in y])
for train_index, test_index in skf.split(file_list, y_sample):
    X_t, X_v = X[train_index], X[test_index]
    y_t, y_v = y[train_index], y[test_index]
    X_train = []
    y_train = []
    for v1 in X_t:
        for v2 in v1:
            X_train.append(v2)
    X_train = np.array(X_train)
    for v1 in y_t:
        for v2 in v1:
            y_train.append(v2)
    y_train = np.array(y_train)
    
    #clf = LinearSVC(random_state=random_state)
    clf = RandomForestClassifier(n_estimators=298, random_state=random_state)
    # clf = XGBClassifier(max_depth=8, learning_rate=0.05, n_estimators=700, seed=random_state)
    #clf = KNeighborsClassifier(algorithm='ball_tree',n_neighbors=3, weights='uniform', p=1)
    clf.fit(X_train, y_train)
    
    pred_array = []
    for i in range(len(X_v)):
        pred = clf.predict(X_v[i])
        pred_array.append(Counter(pred).most_common(1)[0][0])
    pred_array = np.array(pred_array)
    y_valid = np.array([a[0] for a in y_v])
    
    precision += precision_score(y_valid, pred_array, average='macro')
    recall += recall_score(y_valid, pred_array, average='macro')
    f_value += f1_score(y_valid, pred_array, average='macro')

precision = precision / float(n_splits)
recall = recall / float(n_splits)
f_value = f_value / float(n_splits)

In [35]:
f_value

0.47403628117913837

In [13]:
pred_array

array([0, 0, 0, 0, 2, 0, 0, 2, 0, 1, 0, 0, 1, 2, 2, 0, 0, 2])

In [8]:
y_sample

array([ array([ -1.16506500e+01,  -2.17904505e+00,  -4.93832402e-01,
         6.26877868e-01,   5.45670657e-03,  -7.48441120e-01,
         1.24332992e-01,  -2.45808878e-01,  -3.90898699e-01,
         2.77513033e-01,  -2.73682211e-01,   6.80038883e-01,
        -6.53030230e-02]),
       1,
       array([ -1.32480825e+01,   6.13861881e-01,  -7.39003352e-01,
        -1.01390687e+00,   5.06155338e-02,  -9.89755211e-01,
         2.32048599e-03,   1.79234255e-01,   5.45656196e-01,
        -9.80990167e-02,  -1.70532136e-01,  -2.24585425e-01,
         3.52016311e-01]),
       array([ -1.35732394e+01,  -3.15067256e-01,  -3.11660826e-02,
         3.47533606e-01,   2.55674821e-01,  -3.26620420e-01,
         4.20865539e-01,  -3.23893022e-01,  -2.68884129e-01,
         3.69221077e-01,  -3.70329118e-01,  -1.55651574e-03,
        -5.96313422e-01]),
       2, 2,
       array([ -1.27131346e+01,  -1.52155171e+00,  -2.30000696e-01,
         2.63160508e-01,  -4.32260388e-01,  -5.85646239e-01,
        -1.08